# Preparation

## Colab

In [4]:
!pip install transformers
!pip install torch
!pip install emoji

^C
Traceback (most recent call last):
  File "/Users/june/miniconda3/envs/rosetta/bin/pip", line 11, in <module>
    sys.exit(main())
  File "/Users/june/miniconda3/envs/rosetta/lib/python3.8/site-packages/pip/_internal/cli/main.py", line 71, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/Users/june/miniconda3/envs/rosetta/lib/python3.8/site-packages/pip/_internal/commands/__init__.py", line 96, in create_command
    module = importlib.import_module(module_path)
  File "/Users/june/miniconda3/envs/rosetta/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstra

In [3]:
import os
print(os.getcwd())

# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

/Users/june/Documents/Code/FYP/code_data


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd MyDrive/MyDrive/Capstone/code_data

In [ ]:
!pwd

## Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.sampler import WeightedRandomSampler
from torch.optim import lr_scheduler

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.inspection import permutation_importance

from fetchData import fetchdata, cv_events
import __MLP
# from __MLP import getSamplers, convert_df_to_unsqueezed_tensor, train_sequential, clf_report
import random

import __Preprocessing
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

pd.set_option('display.max_columns', None)

In [3]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Data

In [4]:
# Final
pheme_sparse_final = pd.read_csv('./data/_PHEME_sparse.csv')
pheme_y = pd.read_csv('./data/_PHEME_target.csv').target
pheme_pos_final = pd.read_csv('./data/_PHEME_postags.csv')
pheme_thread_final_avg = pd.read_csv('./data/_PHEME_thread_avg.csv')
pheme_thread_final_std = pd.read_csv('./data/_PHEME_thread_std.csv')

ext_pos_final = pd.read_csv('./data/_PHEMEext_postags.csv')
ext_sparse_final = pd.read_csv('./data/_PHEMEext_sparse.csv')
ext_y = pd.read_csv('./data/_PHEMEext_text.csv').target
ext_thread_final_avg = pd.read_csv('./data/_PHEMEext_thread_avg.csv')
ext_thread_final_std = pd.read_csv('./data/_PHEMEext_thread_std.csv')

pheme_bert_simple_normal = pd.read_csv('./data/_PHEME_Bert_final_simple_nrmzd.csv')
ext_bert_simple_normal = pd.read_csv('./data/_PHEMEext_Bert_final_simple_nrmzd.csv')

pheme_bert_brackets_normal = pd.read_csv('./data/_PHEME_Bert_final_brackets_nrmzd.csv')
ext_bert_brackets_normal = pd.read_csv('./data/_PHEMEext_Bert_final_brackets_nrmzd.csv')

pheme_event = pd.read_csv('./data/_PHEME_text.csv')['Event']
ext_event = pd.read_csv('./data/_PHEMEext_text.csv').Event
# pheme_AVGw2v = pd.read_csv('./data/_PHEME_text_AVGw2v.csv').drop(['token'],axis=1)
# ext_AVGw2v = pd.read_csv('./data/_PHEMEext_text_AVGw2v.csv').drop(['token'],axis=1)


In [5]:
pheme_textonly = pd.read_csv('./data/_PHEME_textonly.csv')
ext_textonly = pd.read_csv('./data/_PHEMEext_textonly.csv')


In [6]:

pheme_AVGw2v = pd.read_csv('./data/_PHEME_text_AVGw2v_final.csv')
ext_AVGw2v = pd.read_csv('./data/_PHEMEext_text_AVGw2v_final.csv')

pheme_root = pd.concat([pheme_sparse_final, pheme_pos_final],axis=1)
ext_root = pd.concat([ext_sparse_final, ext_pos_final],axis=1)

pheme_root_thread = pd.concat([pheme_root, pheme_thread_final_avg],axis=1)
ext_root_thread = pd.concat([ext_root, ext_thread_final_avg],axis=1)

pheme_total_bert= pd.concat([pheme_root_thread, pheme_bert_simple_normal],axis=1)
ext_total_bert = pd.concat([ext_root_thread, ext_bert_simple_normal],axis=1)
pheme_total_w2v= pd.concat([pheme_root_thread, pheme_AVGw2v],axis=1)
ext_total_w2v = pd.concat([ext_root_thread, ext_AVGw2v],axis=1)

pheme_w2v_bert = pd.concat([pheme_AVGw2v, pheme_bert_simple_normal],axis=1)
ext_w2v_bert = pd.concat([ext_AVGw2v, ext_bert_simple_normal],axis=1)

In [7]:
all_event = pd.concat([pheme_event,ext_event],axis=0, ignore_index=True)
all_y = pd.concat([pheme_y,ext_y],axis=0, ignore_index=True)

all_root = pd.concat([pheme_root, ext_root],axis=0, ignore_index=True)
all_thread = pd.concat([pheme_thread_final_avg, ext_thread_final_avg],axis=0, ignore_index=True)
all_bert_simple = pd.concat([pheme_bert_simple_normal,ext_bert_simple_normal],axis=0,ignore_index=True)
all_AVGw2v = pd.concat([pheme_AVGw2v,ext_AVGw2v],axis=0,ignore_index=True)
all_w2v_bert = pd.concat([pheme_w2v_bert,ext_w2v_bert],axis=0,ignore_index=True)
all_root_thread = pd.concat([pheme_root_thread,ext_root_thread],axis=0,ignore_index=True)
all_total_bert = pd.concat([pheme_total_bert,ext_total_bert],axis=0,ignore_index=True)
all_total_w2v = pd.concat([pheme_total_w2v,ext_total_w2v],axis=0,ignore_index=True)

In [8]:
print("pheme_sparse_final", pheme_sparse_final.shape)
print(pheme_sparse_final.columns, "\n")
print("pheme_pos_final", pheme_pos_final.shape)
x = zip(pheme_pos_final.columns.values, ['Interjection', 'Hashtag', 'coordinating conjunction', 'punctuation', 'at-mention', 'proper noun', 'adjective' 'determiner' 'other', 'nominal + verbal',
    'common noun', 'pronoun', 'pre/postposition/subordinating conjunction', 'adverb', 'nominal + possessive', 'verb particle', 'URL or email', 'verb incl. copula, auxiliaries',
 'existential there, predeterminers', 'proper noun + possessive', 'discourse marker'])
print(set(x), "\n")
print("pheme_thread_final", pheme_thread_final_avg.shape)
print(pheme_thread_final_avg.columns, "\n")
print("pheme_bert_simple_normal", pheme_bert_simple_normal.shape)
print(pheme_bert_simple_normal.columns, "\n")


pheme_sparse_final (5802, 28)
Index(['emoji_count', 'URLcount', 'has_media', 'Skepticism', 'MentionCount',
       'FirstPersonPronoun', 'SecondPersonPronoun', 'ThirdPersonPronoun',
       'test_auxiliary', 'test_tentat', 'test_certain', 'Numeral',
       'char_count', 'word_count', 'HashTag', 'has_question', 'has_exclaim',
       'has_period', 'capital_ratio', 'retweet_count', 'tweet_count',
       'listed_count', 'friends_count', 'follower_count', 'followers/friend',
       'favourites_count', 'account_age_days', 'verified'],
      dtype='object') 

pheme_pos_final (5802, 21)
{('&', 'coordinating conjunction'), ('u', 'existential there, predeterminers'), ('@', 'at-mention'), ('s', 'URL or email'), ('o', 'adverb'), ('l', 'pronoun'), ('p', 'nominal + possessive'), ('t', 'verb incl. copula, auxiliaries'), ('n', 'pre/postposition/subordinating conjunction'), ('r', 'verb particle'), ('x', 'discourse marker'), ('d', 'nominal + verbal'), ('^', 'proper noun'), ('#', 'Hashtag'), ('g', 'common 

In [9]:
print("pheme_sparse_final", ext_sparse_final.shape)
print(ext_sparse_final.columns, "\n")
print("pheme_pos_final", ext_pos_final.shape)
x = zip(pheme_pos_final.columns.values, ['Interjection', 'Hashtag', 'coordinating conjunction', 'punctuation', 'at-mention', 'proper noun', 'adjective' 'determiner' 'other', 'nominal + verbal',
    'common noun', 'pronoun', 'pre/postposition/subordinating conjunction', 'adverb', 'nominal + possessive', 'verb particle', 'URL or email', 'verb incl. copula, auxiliaries',
 'existential there, predeterminers', 'proper noun + possessive', 'discourse marker'])
print(set(x), "\n")
print("pheme_thread_final", ext_thread_final_avg.shape)
print(ext_thread_final_avg.columns, "\n")
print("pheme_bert_simple_normal", ext_bert_simple_normal.shape)
print(ext_bert_simple_normal.columns, "\n")


pheme_sparse_final (623, 28)
Index(['emoji_count', 'URLcount', 'has_media', 'Skepticism', 'MentionCount',
       'FirstPersonPronoun', 'SecondPersonPronoun', 'ThirdPersonPronoun',
       'test_auxiliary', 'test_tentat', 'test_certain', 'Numeral',
       'char_count', 'word_count', 'HashTag', 'has_question', 'has_exclaim',
       'has_period', 'capital_ratio', 'retweet_count', 'tweet_count',
       'listed_count', 'friends_count', 'follower_count', 'followers/friend',
       'favourites_count', 'account_age_days', 'verified'],
      dtype='object') 

pheme_pos_final (623, 21)
{('&', 'coordinating conjunction'), ('u', 'existential there, predeterminers'), ('@', 'at-mention'), ('s', 'URL or email'), ('o', 'adverb'), ('l', 'pronoun'), ('p', 'nominal + possessive'), ('t', 'verb incl. copula, auxiliaries'), ('n', 'pre/postposition/subordinating conjunction'), ('r', 'verb particle'), ('x', 'discourse marker'), ('d', 'nominal + verbal'), ('^', 'proper noun'), ('#', 'Hashtag'), ('g', 'common no

## Functions

In [10]:
seed = 42
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)


torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False

def _init_fn(worker_id):
    np.random.seed(int(seed))

def test_data_process(X_test, y_test):
    tensor_x1 = torch.Tensor(X_test.values).unsqueeze(1)
    tensor_y1 = torch.Tensor(y_test.values).unsqueeze(1)
    test_dataset = TensorDataset(tensor_x1,tensor_y1)

    batch_size = 8

    # train_sampler, test_sampler = __MLP.getSamplers(pheme_y, tensor_x2)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    data = next(iter(test_dataloader))
    print("mean: %s, std: %s" %(data[0].mean(), data[0].std()))

    test_size = int(tensor_y1.size(0))

    print("Test Size",test_size)

    # predict_batch
    return test_dataloader, test_size


def test_data_process(X_test, y_test):

    tensor_x1 = torch.Tensor(X_test.values).unsqueeze(1)
    tensor_y1 = torch.Tensor(y_test.values).unsqueeze(1)
    test_dataset = TensorDataset(tensor_x1,tensor_y1)

    batch_size = 8

    # train_sampler, test_sampler = __MLP.getSamplers(pheme_y, tensor_x2)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    data = next(iter(test_dataloader))
    print("mean: %s, std: %s" %(data[0].mean(), data[0].std()))

    test_size = int(tensor_y1.size(0))

    print("Test Size",test_size)

    # predict_batch
    return test_dataloader, test_size

def predict(model, criterion, val_dataloader, val_size):
    model.eval()
    val_label_list = []
    # val_preds_list = []
    running_val_preds = []

    with torch.no_grad():
        val_loss = 0.0
        val_corrects = 0
        f1_running = 0
        for j, val in enumerate(val_dataloader, 0):
            val_x, val_label = val
            val_x, val_label = val_x.float(), val_label.float()
            val_outputs = model(val_x)
            val_preds = val_outputs.squeeze(1) > 0.0
            f1_running += (f1_score(val_label, val_preds,zero_division=True) * val_x.size(0))
            v_loss = criterion(val_outputs, val_label.unsqueeze(1))
            val_loss += (v_loss.item() * val_x.size(0))
            val_corrects += torch.sum(val_preds == val_label)
            val_label_list.append(val_label)
            running_val_preds.append(val_preds)

    running_val_preds = torch.cat(running_val_preds, 0)
    val_label_list = torch.cat(val_label_list, 0)
    val_corrects = val_corrects
    val_loss = val_loss/val_size
    val_acc = val_corrects.double().numpy() / val_size
    f1_running /= val_size
    print("accuracy_score:\t\t%.4f" % val_acc)
    print('Precision Score:\t%.4f' % precision_score(val_label_list,running_val_preds))
    print('Recall Score:\t\t%.4f' % recall_score(val_label_list,running_val_preds))
    print("f1_score:\t\t%.4f" % f1_running)
    print("Test_loss:\t\t%.4f" % val_loss)


In [11]:
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False

seed = 42

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [12]:
def getDataSize(tensor_x1, tensor_y1, tensor_x2, tensor_y2):
    train_size = int(tensor_y1.size(0))
    test_size = int(tensor_y2.size(0))

    # print("Variables)\n\tTrain:%s\n\tTest: %s"%(tensor_x1.size(),tensor_x2.size()))
    # print("\tTargets:%s \ %s"%(tensor_y1.size()[0],tensor_y2.size()[0]))
    # print("Train Size",train_size,"Test Size",test_size)
    # print()
    return train_size, test_size

def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
      # print(f'Reset trainable parameters of layer = {layer}')
      # print("Parameter Resetted")
      layer.reset_parameters()

class writeLog():
  def write(self, fileName, text):
    print(text)
    f=open(fileName,'a')
    f.write(text)
    f.write("\n")
    f.close()
  def writeWithoutCR(self, fileName, text):
    f=open(fileName,'a')
    f.write(text)
    f.close()


In [13]:
def cv_process(dataset, criterion, modelClass, target, epochs, events, verbose=True, scaling=False):

    # cv_pd_list[?][0]은 Training cv_pd_list[?][1] Testing
    cv_pd_list = []
    data = pd.concat([dataset, events, target], axis=1)
    NUM_EVENT = data.Event.unique().shape[0]
    EVENTS = data.Event.unique()
    results = {}
    # modelClass.__class__

    for i, d in enumerate(EVENTS):
        df1, df2 = [x for _, x in data.groupby(data['Event'] != d)]
        df1.reset_index(inplace=True, drop=True)
        df2.reset_index(inplace=True, drop=True)
        cv_pd_list.append([df2, df1])
    
    # for train, test in cv_pd_list:
    #     print("Train: %s \ Test: %s" % (train.shape, test.shape))

    log = writeLog()
    modelname = modelClass.__name__
    PREFIX = "./Model/"+modelname+"_"
    log.write(PREFIX+"log.txt",f"\nSTARTING TEST of {epochs} EPOCH\n")

    for index, fold in enumerate(cv_pd_list):

        # DATA PREPARATION
        train, test = fold
        log.writeWithoutCR(PREFIX+"log.txt",f"\n----------------------------------------------------------------------------\n> FOLD {int(index)+1}\n----------------------------------------------------------------------------")
        print(f'> FOLD {int(index)+1}')
        train_target = train.pop('target')
        train.pop('Event').values
        test_target = test.pop('target')
        test.pop('Event')

        if scaling==True:
            scaler = StandardScaler()
            train = pd.DataFrame(scaler.fit_transform(train))
            test = pd.DataFrame(scaler.transform(test))

        tensor_x1, tensor_y1, tensor_x2, tensor_y2 = __MLP.convert_df_to_unsqueezed_tensor(
            train.values, train_target, test.values, test_target.values)
        train_dataset = TensorDataset(tensor_x1, tensor_y1)
        test_dataset = TensorDataset(tensor_x2, tensor_y2)

        batch_size = 8

        # train_sampler, test_sampler = __MLP.getSamplers(train_target, tensor_x2)
        counts = np.bincount(train_target.values)
        labels_weights = 1. / counts
        weights = labels_weights[train_target.values]
        train_sampler = WeightedRandomSampler(weights, len(weights))
        test_sampler = SequentialSampler(tensor_x2)


        train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                                    sampler=train_sampler, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                                    shuffle=False, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)

        data = next(iter(train_dataloader))

        train_size, test_size = getDataSize(tensor_x1, tensor_y1, tensor_x2, tensor_y2)

        if verbose==True:
            print("mean: %s, std: %s" % (data[0].mean(), data[0].std()))
            print("Train Size",train_size,"Test Size",test_size)
        log.writeWithoutCR(PREFIX+"log.txt", f"\nmean: {data[0].mean()}, std: {data[0].std()}")
        log.writeWithoutCR(PREFIX+"log.txt", f"\nTrain Size: {train_size}, Test Size: {test_size}\n")
        

        model = modelClass()
        model.apply(reset_weights)
        # for layer in model.children():
        #     if hasattr(layer, 'reset_parameters'):
        #         print(f'Reset trainable parameters of layer = {layer}')
        #         layer.reset_parameters()


        # model_sparse = sparse_model()
        # criterion = nn.BCEWithLogitsLoss()
        # optimizer = optim.SGD(model_sparse.parameters(), lr=0.01, momentum=0.9)
        # optimizer = optim.Adam(model_sparse.parameters(), lr=5e-5, eps=1e-8, weight_decay=1e-7)
        optimizer = AdamW(model.parameters(),
                        lr=1e-4,    # Default learning rate
                        # lr=9e-5,    # Default learning rate
                        eps=1e-8,    # Default epsilon value
                        weight_decay=1e-6
                        )


        total_steps = len(train_dataloader) * epochs
        # print(f'length of tloader: {len(train_dataloader)}')
        # print(f'total step: {total_steps}')
        log.writeWithoutCR(PREFIX+"log.txt", f'total step: {total_steps}\n')
        
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=0,  # Default value
                                                    num_training_steps=total_steps)

        # print(f'Model: {modelname}')
        PATH = "./Model/"+modelname+"_"+str(index+1)+".pt"
        # print(f'PATH: {PATH}')
        log.writeWithoutCR(PREFIX+"log.txt", f'PATH: {PATH}')

        training_acc = []
        training_loss = []
        # train_acc, train_loss, val_acc, val_loss_list = __MLP.train_sequential(model=model, num_epochs=epochs, patience=patience, criterion=criterion, optimizer=optimizer, scheduler=scheduler, train_loader=train_dataloader, train_size=train_size, test_loader=test_dataloader, test_size=test_size, PATH=PATH)

        # Run the training loop for defined number of epochs
        for epoch in range(0, epochs):

            # Print epoch
            if (verbose!=False):
                # pass
                print(f'Starting epoch {epoch+1}')
            elif (verbose!=True):
                if epoch%50 == 49:
                    print(f'Starting epoch {epoch+1}')
            # Set current loss value
            current_loss = 0.0
            running_corrects = 0.0
            running_loss = 0.0

            # Iterate over the DataLoader for training data
            for i, data in enumerate(train_dataloader, 0):

                # Get inputs
                inputs, targets = data

                # Zero the gradients
                optimizer.zero_grad()

                # Perform forward pass
                outputs = model(inputs)

                outputs = outputs.view(outputs.size(0), -1)

                # Compute Prediction Outputs
                # preds = outputs.squeeze(1) > 0.0
                preds = outputs > 0.0

                # Compute loss
                loss = criterion(outputs, targets)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == targets.data).data

                # Perform backward pass
                loss.backward()

                # Perform optimization and Scheduler
                optimizer.step()
                scheduler.step()

                # Print statistics
                # current_loss += loss.item() # 원본
                # if i % len(train_dataloader) == len(train_dataloader)-1:
                #     print('Loss after mini-batch %5d: %.3f' %
                #           (i + 1, current_loss / i+1))

                current_loss += loss.item() * inputs.size(0)
                if verbose == True:
                    if i % len(train_dataloader) == len(train_dataloader)-1:
                        print("Loss/ACC after mini-batch %5d: %.3f / %.4f" %
                            (i + 1, current_loss / train_size, running_corrects/train_size))

            # epoch_acc = running_corrects.double() / train_size
            epoch_acc = running_corrects / train_size
            epoch_loss = running_loss / train_size
            training_acc.append(epoch_acc)
            training_loss.append(epoch_loss)
            # print('Epoch {}/{}\tTrain) Acc: {:.4f}, Loss: {:.4f}'.format(epoch+1,
                                                                        #  epochs, epoch_acc, epoch_loss))
            
        # Process is complete.
        if verbose==True:
            print('>> Training process has finished. Saving trained model and starting Testing')

        # Print about testing

        # Saving the model
        torch.save(model.state_dict(), PATH)

        # Evaluation for this fold
        correct, total = 0, 0
        val_corrects=0
        f1_batch_epoch = 0
        val_label_list = []
        val_loss = 0
        with torch.no_grad():

            # Iterate over the test data and generate predictions
            for i, data in enumerate(test_dataloader, 0):

                # Get inputs
                inputs, targets = data

                # Generate outputs
                outputs = model(inputs)

                # Set total and correct
                outputs = outputs.view(outputs.size(0), -1).float()
                predicted = (outputs > 0.0).float()
                correct += (predicted == targets).float().sum().item()

                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                #!
                preds = (outputs > 0.0).float()
                # running_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == targets.data).data
                # f1_batch = f1_score(targets.cpu(), outputs.sigmoid().cpu() > 0.5, average='macro')
                f1_batch = f1_score(targets.cpu(), preds, average='macro')
                f1_batch_epoch += f1_batch * inputs.size(0)
                # f1_running += (f1_score(targets, preds, average='macro') * inputs.size(0))
                total += targets.size(0)
            
            if verbose==True:
                # Print accuracy
                print('Accuracy for fold %d: %f %%' % (index, 100.0 * correct / total))
                # print('Accuracy-2 for fold %d: %f %%' % (index, 100.0 * val_corrects / total))
                # print('F1 Score-2 for fold %d: %f ->  %%' %(index, f1_score(targets, preds, zero_division=False)))
                # print('F1 Score-3 for fold %d: %f %%' %(index, f1_score(targets, predicted, zero_division=False)))
                print('F1 Score for fold %d: %f %%' %(index, f1_batch_epoch / total))
                print('Loss for fold %d: %f %%' %(index, val_loss / total))
                # print('F1 Score-5 for fold %d: %f %%' %(index, f1_batch_epoch / test_size))
                # print('F1 Score-6 for fold %d: %f %%' %(index, f1_running / test_size))
            
            results[index] = [100.0 * (correct / total), 100.0 * f1_batch_epoch / total, val_loss / total, train_size, test_size]
            log.writeWithoutCR(PREFIX+"log.txt", f'\nAccuracy for fold {index+1}: {100.0 * correct / total:.3f}')
            log.writeWithoutCR(PREFIX+"log.txt", f'\nF1 Score for fold {index+1}: {f1_batch_epoch / total:.3f}')
            log.writeWithoutCR(PREFIX+"log.txt", f'\nLoss for fold {index+1}: {val_loss / total:.3f}')

    # ---------------------------- Print fold results ---------------------------- #
    # log.write(PREFIX+"log.txt",f"FOLD {int(index)+1}\n----------------------------------------------------------------------------")

    # print(f'K-FOLD CROSS VALIDATION RESULTS FOR {NUM_EVENT} FOLDS')
    log.write(PREFIX+"log.txt", f'\n\n----------------------------------------------------------------------------\n>>>K-FOLD CROSS VALIDATION RESULTS FOR {NUM_EVENT} FOLDS\n----------------------------------------------------------------------------')

    # print('----------------------------------------------------------------------------')
    acc_sum = 0.0
    f1_sum = 0.0
    loss_sum = 0.0
    for key, value in results.items():
        # print(f'Fold {key}: Acc {value[0]}, F1 {value[1]} %')
        log.write(PREFIX+"log.txt", f'Fold {key}: Acc {value[0]}, F1 {value[1]}')
        acc_sum += value[0]
        f1_sum += value[1]
        loss_sum += value[2]
    # print(f'Average: {acc_sum/len(results.items())} %')
    # print(f'F1: {f1_sum/len(results.items())} %')
    log.write(PREFIX+"log.txt", f'Average: {acc_sum/len(results.items()):.3f}%')
    log.write(PREFIX+"log.txt", f'F1: {f1_sum/len(results.items()):.3f}%')
    log.write(PREFIX+"log.txt", f'Loss: {loss_sum/len(results.items()):.3f}\n')

    return results

In [15]:
def cv_process(dataset_embed, dataset_sparse,  criterion, modelClass, target, epochs, events, verbose=True, scaling=False):

    # cv_pd_list[?][0]은 Training cv_pd_list[?][1] Testing
    cv_pd_list_embed = []
    cv_pd_list_sparse = []
    dataEmbed = pd.concat([dataset_embed, events, target], axis=1)
    dataSparse = pd.concat([dataset_sparse, events, target], axis=1)
    NUM_EVENT = dataEmbed.Event.unique().shape[0]
    EVENTS = dataEmbed.Event.unique()
    results = {}
    # modelClass.__class__

    for i, d in enumerate(EVENTS):
        df1, df2 = [x for _, x in dataEmbed.groupby(dataEmbed['Event'] != d)]
        df1.reset_index(inplace=True, drop=True)
        df2.reset_index(inplace=True, drop=True)
        cv_pd_list_embed.append([df2, df1])

    for i, d in enumerate(EVENTS):
        df1, df2 = [x for _, x in dataSparse.groupby(dataSparse['Event'] != d)]
        df1.reset_index(inplace=True, drop=True)
        df2.reset_index(inplace=True, drop=True)
        cv_pd_list_sparse.append([df2, df1])
    
    # for train, test in cv_pd_list:
    #     print("Train: %s \ Test: %s" % (train.shape, test.shape))

    log = writeLog()
    modelname = modelClass.__name__
    PREFIX = "./Model/"+modelname+"_"
    log.write(PREFIX+"log.txt",f"\nSTARTING TEST of {epochs} EPOCH\n")

    for index, fold in enumerate(cv_pd_list_embed):
        fold_2 = cv_pd_list_sparse[index]

        # DATA PREPARATION
        train_embed, test_embed = fold
        train_sparse, test_sparse = fold_2
        log.writeWithoutCR(PREFIX+"log.txt",f"\n----------------------------------------------------------------------------\n> FOLD {int(index)+1}\n----------------------------------------------------------------------------")
        print(f'> FOLD {int(index)+1}')
        train_target = train_embed.pop('target')
        train_sparse.pop('target')
        train_embed.pop('Event').values
        train_sparse.pop('Event').values
        test_target = test_embed.pop('target')
        test_sparse.pop('target')
        test.test_embed('Event')
        test_sparse.pop('Event')

        if scaling==True:
            scaler = StandardScaler()
            train_sparse = pd.DataFrame(scaler.fit_transform(train_sparse))
            test_sparse = pd.DataFrame(scaler.transform(test_sparse))

        tensor_x1, tensor_y1, tensor_x2, tensor_y2 = __MLP.convert_df_to_unsqueezed_tensor(
            train.values, train_target, test.values, test_target.values)
        train_dataset = TensorDataset(tensor_x1, tensor_y1)
        test_dataset = TensorDataset(tensor_x2, tensor_y2)

        batch_size = 8

        # train_sampler, test_sampler = __MLP.getSamplers(train_target, tensor_x2)
        counts = np.bincount(train_target.values)
        labels_weights = 1. / counts
        weights = labels_weights[train_target.values]
        train_sampler = WeightedRandomSampler(weights, len(weights))
        test_sampler = SequentialSampler(tensor_x2)


        train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                                    sampler=train_sampler, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                                    shuffle=False, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)

        data = next(iter(train_dataloader))

        train_size, test_size = getDataSize(tensor_x1, tensor_y1, tensor_x2, tensor_y2)

        if verbose==True:
            print("mean: %s, std: %s" % (data[0].mean(), data[0].std()))
            print("Train Size",train_size,"Test Size",test_size)
        log.writeWithoutCR(PREFIX+"log.txt", f"\nmean: {data[0].mean()}, std: {data[0].std()}")
        log.writeWithoutCR(PREFIX+"log.txt", f"\nTrain Size: {train_size}, Test Size: {test_size}\n")
        

        model = modelClass()
        model.apply(reset_weights)
        # for layer in model.children():
        #     if hasattr(layer, 'reset_parameters'):
        #         print(f'Reset trainable parameters of layer = {layer}')
        #         layer.reset_parameters()


        # model_sparse = sparse_model()
        # criterion = nn.BCEWithLogitsLoss()
        # optimizer = optim.SGD(model_sparse.parameters(), lr=0.01, momentum=0.9)
        # optimizer = optim.Adam(model_sparse.parameters(), lr=5e-5, eps=1e-8, weight_decay=1e-7)
        optimizer = AdamW(model.parameters(),
                        lr=1e-4,    # Default learning rate
                        # lr=9e-5,    # Default learning rate
                        eps=1e-8,    # Default epsilon value
                        weight_decay=1e-6
                        )


        total_steps = len(train_dataloader) * epochs
        # print(f'length of tloader: {len(train_dataloader)}')
        # print(f'total step: {total_steps}')
        log.writeWithoutCR(PREFIX+"log.txt", f'total step: {total_steps}\n')
        
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=0,  # Default value
                                                    num_training_steps=total_steps)

        # print(f'Model: {modelname}')
        PATH = "./Model/"+modelname+"_"+str(index+1)+".pt"
        # print(f'PATH: {PATH}')
        log.writeWithoutCR(PREFIX+"log.txt", f'PATH: {PATH}')

        training_acc = []
        training_loss = []
        # train_acc, train_loss, val_acc, val_loss_list = __MLP.train_sequential(model=model, num_epochs=epochs, patience=patience, criterion=criterion, optimizer=optimizer, scheduler=scheduler, train_loader=train_dataloader, train_size=train_size, test_loader=test_dataloader, test_size=test_size, PATH=PATH)

        # Run the training loop for defined number of epochs
        for epoch in range(0, epochs):

            # Print epoch
            if (verbose!=False):
                # pass
                print(f'Starting epoch {epoch+1}')
            elif (verbose!=True):
                if epoch%50 == 49:
                    print(f'Starting epoch {epoch+1}')
            # Set current loss value
            current_loss = 0.0
            running_corrects = 0.0
            running_loss = 0.0

            # Iterate over the DataLoader for training data
            for i, data in enumerate(train_dataloader, 0):

                # Get inputs
                inputs, targets = data

                # Zero the gradients
                optimizer.zero_grad()

                # Perform forward pass
                outputs = model(inputs)

                outputs = outputs.view(outputs.size(0), -1)

                # Compute Prediction Outputs
                # preds = outputs.squeeze(1) > 0.0
                preds = outputs > 0.0

                # Compute loss
                loss = criterion(outputs, targets)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == targets.data).data

                # Perform backward pass
                loss.backward()

                # Perform optimization and Scheduler
                optimizer.step()
                scheduler.step()

                # Print statistics
                # current_loss += loss.item() # 원본
                # if i % len(train_dataloader) == len(train_dataloader)-1:
                #     print('Loss after mini-batch %5d: %.3f' %
                #           (i + 1, current_loss / i+1))

                current_loss += loss.item() * inputs.size(0)
                if verbose == True:
                    if i % len(train_dataloader) == len(train_dataloader)-1:
                        print("Loss/ACC after mini-batch %5d: %.3f / %.4f" %
                            (i + 1, current_loss / train_size, running_corrects/train_size))

            # epoch_acc = running_corrects.double() / train_size
            epoch_acc = running_corrects / train_size
            epoch_loss = running_loss / train_size
            training_acc.append(epoch_acc)
            training_loss.append(epoch_loss)
            # print('Epoch {}/{}\tTrain) Acc: {:.4f}, Loss: {:.4f}'.format(epoch+1,
                                                                        #  epochs, epoch_acc, epoch_loss))
            
        # Process is complete.
        if verbose==True:
            print('>> Training process has finished. Saving trained model and starting Testing')

        # Print about testing

        # Saving the model
        torch.save(model.state_dict(), PATH)

        # Evaluation for this fold
        correct, total = 0, 0
        val_corrects=0
        f1_batch_epoch = 0
        val_label_list = []
        val_loss = 0
        with torch.no_grad():

            # Iterate over the test data and generate predictions
            for i, data in enumerate(test_dataloader, 0):

                # Get inputs
                inputs, targets = data

                # Generate outputs
                outputs = model(inputs)

                # Set total and correct
                outputs = outputs.view(outputs.size(0), -1).float()
                predicted = (outputs > 0.0).float()
                correct += (predicted == targets).float().sum().item()

                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                #!
                preds = (outputs > 0.0).float()
                # running_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == targets.data).data
                # f1_batch = f1_score(targets.cpu(), outputs.sigmoid().cpu() > 0.5, average='macro')
                f1_batch = f1_score(targets.cpu(), preds, average='macro')
                f1_batch_epoch += f1_batch * inputs.size(0)
                # f1_running += (f1_score(targets, preds, average='macro') * inputs.size(0))
                total += targets.size(0)
            
            if verbose==True:
                # Print accuracy
                print('Accuracy for fold %d: %f %%' % (index, 100.0 * correct / total))
                # print('Accuracy-2 for fold %d: %f %%' % (index, 100.0 * val_corrects / total))
                # print('F1 Score-2 for fold %d: %f ->  %%' %(index, f1_score(targets, preds, zero_division=False)))
                # print('F1 Score-3 for fold %d: %f %%' %(index, f1_score(targets, predicted, zero_division=False)))
                print('F1 Score for fold %d: %f %%' %(index, f1_batch_epoch / total))
                print('Loss for fold %d: %f %%' %(index, val_loss / total))
                # print('F1 Score-5 for fold %d: %f %%' %(index, f1_batch_epoch / test_size))
                # print('F1 Score-6 for fold %d: %f %%' %(index, f1_running / test_size))
            
            results[index] = [100.0 * (correct / total), 100.0 * f1_batch_epoch / total, val_loss / total, train_size, test_size]
            log.writeWithoutCR(PREFIX+"log.txt", f'\nAccuracy for fold {index+1}: {100.0 * correct / total:.3f}')
            log.writeWithoutCR(PREFIX+"log.txt", f'\nF1 Score for fold {index+1}: {f1_batch_epoch / total:.3f}')
            log.writeWithoutCR(PREFIX+"log.txt", f'\nLoss for fold {index+1}: {val_loss / total:.3f}')

    # ---------------------------- Print fold results ---------------------------- #
    # log.write(PREFIX+"log.txt",f"FOLD {int(index)+1}\n----------------------------------------------------------------------------")

    # print(f'K-FOLD CROSS VALIDATION RESULTS FOR {NUM_EVENT} FOLDS')
    log.write(PREFIX+"log.txt", f'\n\n----------------------------------------------------------------------------\n>>>K-FOLD CROSS VALIDATION RESULTS FOR {NUM_EVENT} FOLDS\n----------------------------------------------------------------------------')

    # print('----------------------------------------------------------------------------')
    acc_sum = 0.0
    f1_sum = 0.0
    loss_sum = 0.0
    for key, value in results.items():
        # print(f'Fold {key}: Acc {value[0]}, F1 {value[1]} %')
        log.write(PREFIX+"log.txt", f'Fold {key}: Acc {value[0]}, F1 {value[1]}')
        acc_sum += value[0]
        f1_sum += value[1]
        loss_sum += value[2]
    # print(f'Average: {acc_sum/len(results.items())} %')
    # print(f'F1: {f1_sum/len(results.items())} %')
    log.write(PREFIX+"log.txt", f'Average: {acc_sum/len(results.items()):.3f}%')
    log.write(PREFIX+"log.txt", f'F1: {f1_sum/len(results.items()):.3f}%')
    log.write(PREFIX+"log.txt", f'Loss: {loss_sum/len(results.items()):.3f}\n')

    return results

In [12]:
def epochs_diff(testing_results):
    string = ""
    print('----------------------------------------------------------------------------')
    for loop in testing_results:
        acc_sum = 0.0
        f1_sum = 0.0
        loss=0.0
        for key, value in loop.items():
            print(f'Fold {key}: Acc {value[0]:.2f}%, F1 {value[1]:.2f}%, Loss {value[2]:.2f}')
            acc_sum += value[0]
            f1_sum += value[1]
            loss += value[2]
        string+=f'Average: {acc_sum/len(loop.items()):.2f}%\n'
        string+=f'F1: {f1_sum/len(loop.items()):.2f}%\n'
        string+=f'Loss: {loss/len(loop.items()):.2f}\n'
        # print(f'Average: {acc_sum/len(loop.items()):.2f} %')
        # print(f'F1: {f1_sum/len(loop.items()):.2f} %')
        print("-----------------------------")
        return string

# Batch

## CNN

In [25]:
from transformers import AutoModel, AutoTokenizer


In [14]:
class InputExample(object):
    def __init__(self, id, text, labels=None):
        self.id = id
        self.text = text
        self.labels = labels

class InputFeatures(object):
    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids

def get_train_examples(train_file):
    train_df = pd.read_csv(train_file)
    ids = train_df['id'].values
    text = train_df['comment_text'].values
    labels = train_df[train_df.columns[2:]].values
    examples = []
    for i in range(len(train_df)):
        examples.append(InputExample(ids[i], text[i], labels=labels[i]))
    return examples



In [100]:
def get_features_from_examples(examples, max_seq_len, tokenizer):
    features = []
    for i, example in enumerate(examples):
        tokens = tokenizer.tokenize(examples.text[i])
        if len(tokens) > max_seq_len - 2:
            tokens = tokens[:(max_seq_len - 2)]
        tokens = ["[CLS]"] + tokens + ["[SEP]"]
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        segment_ids = [0] * len(tokens)
        padding = [0] * (max_seq_len - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding
        assert len(input_ids) == max_seq_len
        assert len(input_mask) == max_seq_len
        assert len(segment_ids) == max_seq_len
        label_ids = [float(target) for target in examples.target]
        features.append(InputFeatures(input_ids=input_ids,
                                      input_mask=input_mask,
                                      segment_ids=segment_ids,
                                      label_ids=label_ids))
    return features

def get_dataset_from_features(features):
    input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.float)
    dataset = TensorDataset(input_ids,
                            input_mask,
                            segment_ids,
                            label_ids)
    return dataset

def text_preprocessing(text): # Create a function to tokenize a set of texts

    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """

    # text = re.sub(r"http\S+", "*", text)  # http link -> '*'
    # sent = re.sub(r'([^\s\w@#\*]|_)+', '', sent) # Erasing Special Characters

    text = emoji.demojize(text)
    text = re.sub(r':[^:\s]*:', r' \g<0>', text)  # http link -> '*'

    # text = re.sub(r':[^:\s]*(?:::[^:\s]*)*:', r' \g<0> ', text)  # http link -> '*'

    # text = re.sub(r"\n", " ", text)   # mention -> '@'
    text = re.sub(r"http\S+", "HTTPURL", text)  # http link -> '*'
    # text = re.sub(r"@\S+", "@USER", text)   # mention -> '@'

    text = re.sub(r"@[A-Za-z0-9]+", "@USER", text)   # mention -> '@'

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'&amp;', '&', text)
    # text = tweetTokenizer.tokenize(text)
    # text = [emoji.demojize(token) for token in text]

    return text

In [48]:
class KimCNN(nn.Module):
    def __init__(self, embed_num, embed_dim, dropout=0.1, kernel_num=3, kernel_sizes=[2,3,4], num_labels=2):
        super().__init__()
        self.num_labels = num_labels
        self.embed_num = embed_num
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.kernel_num = kernel_num
        self.kernel_sizes = kernel_sizes
        self.embed = nn.Embedding(self.embed_num, self.embed_dim)
        self.convs = nn.ModuleList([nn.Conv2d(1, self.kernel_num, (k, self.embed_dim)) for k in self.kernel_sizes])
        self.dropout = nn.Dropout(self.dropout)
        self.classifier = nn.Linear(len(self.kernel_sizes)*self.kernel_num, self.num_labels)
        
    def forward(self, inputs, labels=None):
        output = inputs.unsqueeze(1)
        output = [nn.functional.relu(conv(output)).squeeze(3) for conv in self.convs]
        output = [nn.functional.max_pool1d(i, i.size(2)).squeeze(2) for i in output]
        output = torch.cat(output, 1)
        output = self.dropout(output)
        logits = self.classifier(output)
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
            return loss
        else:
            return logits


In [49]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)


In [50]:
device = torch.device(type='cuda')
# pretrained_weights = "vinai/bertweet-base"
basemodel = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)
# basemodel = AutoModel.from_pretrained(pretrained_weights)
# basemodel.to(device)


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [77]:
train_input = pd.concat([pheme_textonly, pheme_y],axis=1)

In [76]:
train_input

,text,target
0,BREAKING: Armed man takes hostage in kosher gr...,1
1,"#CharlieHebdo killers dead, confirmed by genda...",1
2,"Top French cartoonists Charb, Cabu, Wolinski, ...",1
3,Police have surrounded the area where the #Cha...,1
4,PHOTO: Armed gunmen face police officers near ...,1
...,...,...
5797,'I'll ride with you' http://t.co/llZnuCAzg5 Au...,0
5798,Canada's thoughts and prayers are with our Aus...,0
5799,Every non-muslim in the world must watch this ...,0
5800,Suspect in Sydney cafe siege identified as Man...,0


In [95]:
features = []
for i ,example in enumerate(train_input):
    # print(train_input.iloc[i])
    tokens = tokenizer.tokenize(example.text)
    # if len(tokens) > max_seq_len - 2:
    #     tokens = tokens[:(max_seq_len - 2)]

AttributeError: 'str' object has no attribute 'text'

In [99]:
train_features = get_features_from_examples(train_input, seq_len, tokenizer)


AttributeError: 'str' object has no attribute 'text'

In [35]:
seq_len = 256
train_file = './data/_PHEME_textonly.csv'
train_examples = get_train_examples(train_file)
train_features = get_features_from_examples(train_examples, seq_len, tokenizer)
train_dataset = get_dataset_from_features(train_features)


train_val_split = 0.1
train_size = int(len(train_dataset)*(1-train_val_split))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

batch = 8
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch)
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch)

KeyError: 'id'

## Trainer

In [111]:
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)

In [ ]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

## Class

In [101]:
from datetime import datetime
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False

In [1]:
# 49-16-5-1
# 49-12-4-1
class root_model(nn.Module):
    def __init__(self):
        super(root_model, self).__init__()  # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(49, 8),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(8, 3),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(3, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

class w2v_model(nn.Module):
    def __init__(self):
        super(w2v_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Linear(200, 36, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(36, 8, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

class bert_model(nn.Module):
    def __init__(self):
        super(bert_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(768, 50, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(50, 8, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x
        
#101-36-1
class all_sparse_model(nn.Module):
    def __init__(self):
        super(all_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Linear(101, 24, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(24, 6, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(6, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

class bert_w2v_model(nn.Module):
    def __init__(self):
        super(bert_w2v_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(968, 64, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            # nn.Linear(100, 12, bias=True),
            # nn.ELU(),
            # nn.Dropout(0.5),
            nn.Linear(64, 12),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(12, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x


class bert_sparse_model(nn.Module):
    def __init__(self):
        super(bert_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Linear(869, 72, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(72, 10, bias=True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(10, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

class bert_sparse_model(nn.Module):
    def __init__(self):
        super(bert_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Linear(869, 80, bias = True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(80, 16, bias = True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(16, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

class w2v_sparse_model(nn.Module):
    def __init__(self):
        super(w2v_sparse_model, self).__init__() # 1*20
        self.layers = nn.Sequential(
            nn.Linear(301, 31, bias = True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(31, 4, bias = True),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(4, 1)
        )
    def forward(self, x):
        x = self.layers(x)
        return x

NameError: name 'nn' is not defined

## Batch 5



In [103]:
result_list_5 = []
result_mean_list_5 = []
criterion = nn.BCEWithLogitsLoss()
testing_epochs = [100]

### RPT + BERT MULTI


In [ ]:
# ---------------------------------------------------------------------------- #
#                                 ALL Features                                 #
# ---------------------------------------------------------------------------- #

dataset1 = all_bert_simple
dataset2 = all_root_thread
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [5]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in bert_sparse_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=bert_sparse_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

epochs_diff(testing_results)
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RPT + BERT'
result_df['CV'] = 9
layers = [module for module in bert_sparse_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = layers

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list_5.append(CV_result_df)

### ROOT + POS  

In [ ]:
# ---------------------------------------------------------------------------- #
#                                  ROOT + POS                                  #
# ---------------------------------------------------------------------------- #

dataset = all_root

testing_epochs = [2]
testing_epochs = testing_epochs
testing_results = []

writeLog().write("./Model/"+root_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+root_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
# writeLog().write("./Model/"+root_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in root_model().children()]}")

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=root_model, events=all_event, target=all_y, epochs=epoch, verbose=False, scaling=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+root_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RP'
result_df['CV'] = 9
layers = [module for module in root_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]
result_mean_list_5.append(CV_result_df)

### THREAD

In [ ]:
# ---------------------------------------------------------------------------- #
#                                    THREAD                                    #
# ---------------------------------------------------------------------------- #

dataset = all_thread

testing_epochs = [5]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+thread_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+thread_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+thread_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in thread_model().children()]}")

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=thread_model, events=all_event, target=all_y, epochs=epoch, verbose=False, scaling=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+thread_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'Thread'
result_df['CV'] = 9
layers = [module for module in thread_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list_5.append(CV_result_df)

### BERT

In [ ]:
# ---------------------------------------------------------------------------- #
#                                     BERT                                     #
# ---------------------------------------------------------------------------- #

dataset = all_bert_simple
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [5]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+bert_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+bert_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+bert_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in bert_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=bert_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+all_bert_simple.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'BERT'
result_df['CV'] = 9
layers = [module for module in bert_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list_5.append(CV_result_df)

### W2V

In [ ]:
# ---------------------------------------------------------------------------- #
#                                     W2V                                      #
# ---------------------------------------------------------------------------- #

dataset = all_AVGw2v
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [50]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+w2v_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+w2v_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+w2v_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in w2v_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=w2v_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+w2v_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'W2V'
result_df['CV'] = 9
layers = [module for module in w2v_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list_5.append(CV_result_df)

### BERT + W2V

In [64]:
# ---------------------------------------------------------------------------- #
#                                  BERT + W2V                                  #
# ---------------------------------------------------------------------------- #

dataset = all_w2v_bert
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [50]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+bert_w2v_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+bert_w2v_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+bert_w2v_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in bert_w2v_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=bert_w2v_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+bert_w2v_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'BERT + W2V'
result_df['CV'] = 9
layers = [module for module in bert_w2v_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list_5.append(CV_result_df)



************************************************************
STARTING A CROSS-VALIDATION ... [50] epochs
************************************************************
TIME: 01/04/2021 12:38:57
LAYER: [Sequential(
  (0): Linear(in_features=968, out_features=64, bias=True)
  (1): ELU(alpha=1.0)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=64, out_features=12, bias=True)
  (4): ELU(alpha=1.0)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=12, out_features=1, bias=True)
)]

STARTING TEST of 50 EPOCH

> FOLD 1
Starting epoch 50
> FOLD 2
Starting epoch 50
> FOLD 3
Starting epoch 50
> FOLD 4
Starting epoch 50
> FOLD 5
Starting epoch 50
> FOLD 6
Starting epoch 50
> FOLD 7
Starting epoch 50
> FOLD 8
Starting epoch 50
> FOLD 9
Starting epoch 50


----------------------------------------------------------------------------
>>>K-FOLD CROSS VALIDATION RESULTS FOR 9 FOLDS
----------------------------------------------------------------------------
Fold 0: Acc 76.57

### ROOT + POS + THREAD

In [ ]:
# ---------------------------------------------------------------------------- #
#                              ROOT + POS + THREAD                             #
# ---------------------------------------------------------------------------- #

dataset = all_root_thread
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [100]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+all_sparse_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+all_sparse_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+all_sparse_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in all_sparse_model().children()]}")

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=all_sparse_model, events=all_event, target=all_y, epochs=epoch, verbose=False, scaling=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+all_sparse_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RPT'
result_df['CV'] = 9
layers = [module for module in all_sparse_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list_5.append(CV_result_df)

### RPT + W2V

In [114]:
# ---------------------------------------------------------------------------- #
#                                 ALL Features                                 #
# ---------------------------------------------------------------------------- #

dataset = all_total_w2v

testing_epochs = [50]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+w2v_sparse_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+w2v_sparse_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+w2v_sparse_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in w2v_sparse_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=w2v_sparse_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

epochs_diff(testing_results)
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RPT + W2V'
result_df['CV'] = 9
layers = [module for module in w2v_sparse_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list_5.append(CV_result_df)



************************************************************
STARTING A CROSS-VALIDATION ... [50] epochs
************************************************************
TIME: 01/04/2021 13:34:25
LAYER: [Sequential(
  (0): Linear(in_features=301, out_features=31, bias=True)
  (1): ELU(alpha=1.0)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=31, out_features=4, bias=True)
  (4): ELU(alpha=1.0)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4, out_features=1, bias=True)
)]

STARTING TEST of 50 EPOCH

> FOLD 1
Starting epoch 50
> FOLD 2
Starting epoch 50
> FOLD 3


### RPT + BERT

In [108]:
# ---------------------------------------------------------------------------- #
#                                 ALL Features                                 #
# ---------------------------------------------------------------------------- #

dataset = all_total_bert
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [5]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in bert_sparse_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=bert_sparse_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

epochs_diff(testing_results)
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RPT + BERT'
result_df['CV'] = 9
layers = [module for module in bert_sparse_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = layers

result_list_5.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list_5.append(CV_result_df)



************************************************************
STARTING A CROSS-VALIDATION ... [5] epochs
************************************************************
TIME: 01/04/2021 13:24:27
LAYER: [Sequential(
  (0): Linear(in_features=869, out_features=80, bias=True)
  (1): ELU(alpha=1.0)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=80, out_features=16, bias=True)
  (4): ELU(alpha=1.0)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=16, out_features=1, bias=True)
)]

STARTING TEST of 5 EPOCH

> FOLD 1


KeyboardInterrupt: 

## Batch 9



In [ ]:
# result_list = []
# result_mean_list = []
criterion = nn.BCEWithLogitsLoss()
testing_epochs = [50]


### ROOT + POS  

In [ ]:
# ---------------------------------------------------------------------------- #
#                                  ROOT + POS                                  #
# ---------------------------------------------------------------------------- #

dataset = all_root

testing_epochs = [2]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+root_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+root_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
# writeLog().write("./Model/"+root_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in root_model().children()]}")

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=root_model, events=all_event, target=all_y, epochs=epoch, verbose=False, scaling=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+root_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RP'
result_df['CV'] = 9
layers = [module for module in root_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]
result_mean_list.append(CV_result_df)

### THREAD

In [ ]:
# ---------------------------------------------------------------------------- #
#                                    THREAD                                    #
# ---------------------------------------------------------------------------- #

dataset = all_thread

testing_epochs = [5]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+thread_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+thread_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+thread_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in thread_model().children()]}")

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=thread_model, events=all_event, target=all_y, epochs=epoch, verbose=False, scaling=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+thread_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'Thread'
result_df['CV'] = 9
layers = [module for module in thread_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list.append(CV_result_df)

### BERT

In [ ]:
# ---------------------------------------------------------------------------- #
#                                     BERT                                     #
# ---------------------------------------------------------------------------- #

dataset = all_bert_simple
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [5]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+bert_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+bert_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+bert_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in bert_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=bert_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+all_bert_simple.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'BERT'
result_df['CV'] = 9
layers = [module for module in bert_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list.append(CV_result_df)

### W2V

In [ ]:
# ---------------------------------------------------------------------------- #
#                                     W2V                                      #
# ---------------------------------------------------------------------------- #

dataset = all_AVGw2v
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [50]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+w2v_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+w2v_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+w2v_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in w2v_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=w2v_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+w2v_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'W2V'
result_df['CV'] = 9
layers = [module for module in w2v_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list.append(CV_result_df)

### BERT + W2V

In [ ]:
# ---------------------------------------------------------------------------- #
#                                  BERT + W2V                                  #
# ---------------------------------------------------------------------------- #

dataset = all_w2v_bert
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [50]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+bert_w2v_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+bert_w2v_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+bert_w2v_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in bert_w2v_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=bert_w2v_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+bert_w2v_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'BERT + W2V'
result_df['CV'] = 9
layers = [module for module in bert_w2v_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list.append(CV_result_df)

### ROOT + POS + THREAD

In [ ]:
# ---------------------------------------------------------------------------- #
#                              ROOT + POS + THREAD                             #
# ---------------------------------------------------------------------------- #

dataset = all_root_thread
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [100]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+all_sparse_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+all_sparse_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+all_sparse_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in all_sparse_model().children()]}")

for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=all_sparse_model, events=all_event, target=all_y, epochs=epoch, verbose=False, scaling=False)
    testing_results.append(result)

writeLog().writeWithoutCR("./Model/"+all_sparse_model.__name__+"_"+"log.txt",f"RESULT:\n{epochs_diff(testing_results)}")
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RPT'
result_df['CV'] = 9
layers = [module for module in all_sparse_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list.append(CV_result_df)

### RPT + W2V

In [ ]:
# ---------------------------------------------------------------------------- #
#                                 ALL Features                                 #
# ---------------------------------------------------------------------------- #

dataset = all_total_w2v

testing_epochs = [50]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+w2v_sparse_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+w2v_sparse_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+w2v_sparse_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in w2v_sparse_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=w2v_sparse_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

epochs_diff(testing_results)
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RPT + W2V'
result_df['CV'] = 9
layers = [module for module in w2v_sparse_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = str(layers)

result_list.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list.append(CV_result_df)

### RPT + BERT

In [ ]:
# ---------------------------------------------------------------------------- #
#                                 ALL Features                                 #
# ---------------------------------------------------------------------------- #

dataset = all_total_bert
criterion = nn.BCEWithLogitsLoss()

testing_epochs = [5]
testing_epochs = testing_epochs

testing_results = []

writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"\n\n************************************************************\nSTARTING A CROSS-VALIDATION ... {testing_epochs} epochs\n************************************************************")
writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"TIME: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
writeLog().write("./Model/"+bert_sparse_model.__name__+"_"+"log.txt",f"LAYER: {[layer for layer in bert_sparse_model().children()]}")


for epoch in testing_epochs:
    result = cv_process(dataset, criterion, modelClass=bert_sparse_model, events=all_event, target=all_y, epochs=epoch, verbose=False)
    testing_results.append(result)

epochs_diff(testing_results)
result_df = pd.DataFrame.from_dict(testing_results[0], orient='index', columns=['Acc', 'F1', 'Loss', 'Train_size','Test_size'])
result_df['Features'] = 'RPT + BERT'
result_df['CV'] = 9
layers = [module for module in bert_sparse_model().modules() if not isinstance(module, nn.Sequential)]
layers = [layer.in_features for layer in layers[1:] if str(layer).startswith("Linear")]
result_df['layers'] = layers

result_list.append(result_df)
CV_result_df = pd.DataFrame(result_df[['Acc','F1', 'Loss', 'CV']].mean(), columns=[result_df.Features[0]]).T
CV_result_df['layers'] = result_df['layers'][0]

result_mean_list.append(CV_result_df)

## RESULT

In [106]:
# ---------------------------------------------------------------------------- #
#                               RESULT OF BATCH 5                              #
# ---------------------------------------------------------------------------- #

# final_result_5 = pd.concat(result_list_5)
# final_result_5.to_csv('./data/final_result_5.csv')
final_result_5 = pd.read_csv('./data/final_result_5.csv', index_col=['Unnamed: 0'])
# final_cv_5 = pd.concat(result_mean_list_5)
# final_cv_5.to_csv('./data/final_cv_5.csv')
final_cv_5 = pd.read_csv('./data/final_cv_5.csv', index_col=['Unnamed: 0'])
final_cv_5

FileNotFoundError: [Errno 2] No such file or directory: './data/final_result_5.csv'

In [ ]:
final_result_5

In [109]:
# final_result = pd.concat(result_list)
# final_result.to_csv('./data/final_result.csv')
final_result = pd.read_csv('./data/final_result.csv', index_col=['Unnamed: 0'])
# final_cv = pd.concat(result_mean_list)
# final_cv.to_csv('./data/final_cv.csv')
final_cv = pd.read_csv('./data/final_cv.csv', index_col=['Unnamed: 0'])
final_cv

,Acc,F1,Loss,CV,layers
W2V,65.783691,44.733944,0.664950,9.0,"[200, 36, 8]"
BERT + W2V,66.369984,45.486365,0.696974,9.0,"[968, 80, 12]"
BERT + W2V,64.828757,43.531413,0.686351,9.0,"[968, 80, 12, 4]"
BERT + W2V,65.697599,45.238628,0.726546,9.0,"[968, 64, 12]"
RPT + W2V,52.371971,33.838737,1.481236,9.0,"[301, 48, 8]"
RPT + W2V,54.028256,34.706857,1.465343,9.0,"[301, 52, 12]"
RPT + W2V,49.238179,32.481797,1.843140,9.0,"[301, 52]"
RPT + W2V,48.116611,31.456886,0.859566,9.0,"[301, 30, 5]"


In [110]:
# final_result